# Text Classification

- Pandas Documentation: http://pandas.pydata.org/
- Scikit Learn Documentation: http://scikit-learn.org/stable/documentation.html
- Seaborn Documentation: http://seaborn.pydata.org/
- Keras Documentation: https://keras.io


In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## Text classification

Our goal is to perform a binary classification on text data. We will perform both a Spam detection example and a Sentiment analysis example. We will attempt 3 strategies:

1) build naive features based on our ideas

2) use well tested feature extraction technique

3) use deep learning and recurrent models on text

### 1. Spam detection on SMS messages

In [ ]:
df = pd.read_csv('../data/sms.tsv', sep='\t')
df.head()

In [ ]:
df['label'].value_counts() / len(df)

### Exercise1: Encode Labels to 0 and 1

Create a variable called y that contains 0 for HAM messages and 1 for SPAM messages. There are several ways to do this.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y = le.fit_transform(df['label'])

In [ ]:
le.classes_

In [ ]:
y

In [ ]:
(df['label'] == 'spam').astype(int)

In [ ]:
df['label'].map({'ham': 0, 'spam': 1})

### Exercise 2: Build naive features based on keywords

- turn all your sms messages to lowercase
- define a function to count occurrences of a single keyword with the following signature:

        def count_word(word, sentence):
            ....
            return count_word_in_sentence
            
            
- to test your function, try it on these examples and check that the results match:
   
        count_word("the", "quick brown fox") # -> 0
        count_word("fox", "quick brown fox") # -> 1
        count_word("a", "a b a abab") # -> 2
     

- using the function `count_word` you just wrote, create a feature matrix `X` using counts of some keywords of your choice. (this will a bag-of-words representation.)
- create other similar features. You could use:
    - the length of the message
    - the presence of numbers
    - the presence of special characters
    - ...

In [ ]:
docs_lower  = df['msg'].str.lower().values

In [ ]:
df['msg'].apply(lambda x: x.lower()).values

In [ ]:
docs = df['msg'].values

In [ ]:
docs_lower = [d.lower() for d in docs]

In [ ]:
def count_word(word, sentence):
    tokens = sentence.split()
    return len([w for w in tokens if w == word])

In [ ]:
X = pd.DataFrame([count_word('free', d) for d in docs_lower],
                 columns=['free'])

for keyword in ['win', 'discount', 'call']:
    X[keyword] = [count_word(keyword, d) for d in docs_lower]
    
X.head()

In [ ]:
import re

In [ ]:
def count_numbers(sentence):
    return len(re.findall('[0-9]', sentence))

In [ ]:
X['num_char'] = [count_numbers(d) for d in docs_lower]

In [ ]:
X.head()

### Exercise 3: Train first model and evaluate performance

- split data in to train and test sets with `test_size=0.3, random_state=0`. you can use the `train_test_split` function from sklearn, which we have used in previous labs
- train model of your choice on these features
- evaluate performance on training and test set
- discuss with classmate:
    - how did you evaluate performance?
    - is model overfitting?
    - is model better than benchmark?

First model:

- 4 keywords: baseline: 0.868, train_score: 0.904, test_score: 0.906
- 4 kwds + digits: baseline: 0.868, train_score: 0.977, test_score: 0.972

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X.values, y, test_size=0.3, random_state=0)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(10, input_shape=(5, ), activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, verbose=0)

train_score = model.evaluate(X_train, y_train, verbose=0)[1]
test_score = model.evaluate(X_test, y_test, verbose=0)[1]
baseline = (pd.Series(y_test).value_counts() / len(y_test))[0]
print("baseline: {:0.3}, train_score: {:0.3}, test_score: {:0.3}".format(baseline, train_score, test_score))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, n_jobs=-1)
model.fit(X_train, y_train)

train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
baseline = (pd.Series(y_test).value_counts() / len(y_test))[0]
print("baseline: {:0.3}, train_score: {:0.3}, test_score: {:0.3}".format(baseline, train_score, test_score))

### Exercise 4: Cross Validation

- perform a 5-Fold cross validation on your model. you can refer back to lab 8 to refresh your memory on how to do this.
- print the confusion matrix and the classification report on the test data

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(model, X, y, cv = 5, n_jobs=-1)
scores

In [ ]:
print("Average score: {:0.3} +/- {:0.3}".format(scores.mean(), scores.std()))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

pd.DataFrame(cm,
             index=['ham', 'spam'],
             columns=['pred_ham', 'pred_spam'])

### Exercise 5: Count Features

- use features based on word counts using the `CountVectorizer` class from Scikit Learn
- use the following function to simplify your code (it encapsulates model training and evaluation):


    def split_fit_eval(X, y, model=None, epochs=10, random_state=0):
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state)

        if not model:
            model = Sequential()
            model.add(Dense(1, input_dim=X.shape[1], activation='sigmoid'))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])

        h = model.fit(X_train, y_train, epochs=epochs, verbose=1)

        train_loss, train_acc = model.evaluate(X_train, y_train)
        test_loss, test_acc = model.evaluate(X_test, y_test)

        return train_loss, train_acc, test_loss, test_acc, model, h


- did you improve the performance?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vocab_size = 3000

In [ ]:
vect = CountVectorizer(decode_error='ignore',
                       stop_words='english',
                       lowercase=True,
                       max_features=vocab_size)

In [ ]:
X = vect.fit_transform(docs)

In [ ]:
Xd = X.todense()

In [ ]:
vocab = vect.get_feature_names()

In [ ]:
vocab[:10]

In [ ]:
vocab[-10:]

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [ ]:
def split_fit_eval(X, y, model=None, epochs=10, random_state=0):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state)
    
    if not model:
        model = Sequential()
        model.add(Dense(1, input_dim=X.shape[1], activation='sigmoid'))

        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    
    h = model.fit(X_train, y_train, epochs=epochs, verbose=1)
    
    train_loss, train_acc = model.evaluate(X_train, y_train)
    test_loss, test_acc = model.evaluate(X_test, y_test)
    
    return train_loss, train_acc, test_loss, test_acc, model, h

In [ ]:
res = split_fit_eval(Xd, y)

In [ ]:
res[1]

In [ ]:
res[3]

In [ ]:
w, b = res[4].get_weights()

In [ ]:
w.ravel()

In [ ]:
feature_weights = pd.Series(w.ravel(), index=vocab).sort_values()

In [ ]:
feature_weights.head(10)

In [ ]:
feature_weights.tail(10)

## Sentiment Analysis

The previous dataset was easy. Let's switch to a harder one and do sentiment analysis on it.

In [ ]:
df = pd.read_csv('../data/rt_critics.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df['fresh'].value_counts() / len(df)

In [ ]:
df = df[df.fresh != 'none'].copy()
df['fresh'].value_counts() / len(df)

In [ ]:
y = le.fit_transform(df['fresh'])

### Exercise 6: TFIDF

- Build features with word frequencies (Tfidf). (sklearn has a preprocessor for this.)
- do train/test split
- train and evaluate a model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vect = TfidfVectorizer(decode_error='ignore',
                       stop_words='english',
                       max_features=20000)

X = vect.fit_transform(df['quote'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

In [ ]:
model = RandomForestClassifier(n_estimators=100, n_jobs=-1)
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

### Exercise 7: NLP with deep learning

- Use the Tokenizer from tensorflow.keras to:
    - Create a vocabulary
    - Convert sentences to sequences of integers
- pad the sequences so that they look like a tensor using the pad_sequences function from tensorflow.keras.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=20000)

docs = df['quote']
tokenizer.fit_on_texts(docs)
sequences = tokenizer.texts_to_sequences(docs)

- check max word index
- check max sequence length

In [ ]:
max_features = max([max(seq) for seq in sequences if len(seq) > 0]) + 1
max_features

In [ ]:
maxlen = max([len(seq) for seq in sequences])
maxlen

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
X = pad_sequences(sequences, maxlen=maxlen)

### Train / Test split on sequences

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

### Exercise 8: Build recurrent neural network model
- use what you have learned to build a recurrent model that classifies the sentiment

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding
from tensorflow.keras.layers import LSTM, GRU

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_features,
                    output_dim=32,
                    input_length=maxlen))
model.add(LSTM(32))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
h = model.fit(X_train, y_train, batch_size=64, epochs=4, validation_split=0.1)

In [ ]:
model.evaluate(X_train, y_train, batch_size=32)


In [ ]:
loss, acc = model.evaluate(X_test, y_test, batch_size=32)
acc

In [ ]:
pd.DataFrame(h.history).plot(ylim=(-0.05, 1.05))

### Exercise 9

- Try changing the network architecture and re-train the model at each change. Can you avoid overfitting?
    - change the number of nodes in the LSTM layer
    - change the output dimension of the Embedding layer
    - add dropout and recurrent dropout to the LSTM
    - add a second LSTM layer
    - add kernel regularizers

In [ ]:
from tensorflow.keras import regularizers

model = Sequential()
model.add(Embedding(input_dim=max_features,
                    output_dim=32,
                    input_length=maxlen))
model.add(LSTM(16, return_sequences=True, dropout=0.1))
model.add(LSTM(8, activity_regularizer=regularizers.l2(0.01), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras import callbacks
h = model.fit(X_train, y_train, batch_size=64, epochs=4, validation_split=0.1, callbacks=[callbacks.EarlyStopping()])

In [ ]:
model.evaluate(X_train, y_train, batch_size=64)

In [ ]:
loss, acc = model.evaluate(X_test, y_test, batch_size=64)
acc